## 177 Null Count

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
from mv_wg_avg import exp_weight_avg


key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [3]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)

df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
# df_new   = utils.reduce_mem_usage(df_new  )

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = utils.reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)


df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7])
# df_new['yyyymm'] =  df_new['purchase_date'].map(lambda x: str(x)[:7])
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)
# df_new = df_new.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
# df_new = []

100%|██████████| 3/3 [00:00<00:00,  3.38it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1971.22 Mb (40.8% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(29112361, 15)
(29112361, 16)


In [4]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df_new = df.copy()
del df
gc.collect()

df = df_hist
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,yyyymm,org_month_lag
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2017-06,12
1,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-8,-0.737967,2017-06-01 22:02:56,1.0,16,37,2017-06,12
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.741648,2017-06-08 18:02:29,1.0,16,37,2017-06,12
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-8,-0.738192,2017-06-14 07:40:48,1.0,16,34,2017-06,12
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-8,-0.726772,2017-06-23 21:43:17,1.0,16,37,2017-06,12


In [7]:
# 期間を絞る
# term
term_num = 13
base_list = np.zeros(term_num)
base_list = np.arange(1, term_num, 1)
last_list = np.arange(3, term_num, 1)
col_month = 'org_month_lag'

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for base_month, last_month in zip(tqdm(base_list), last_list):
    prefix = f'177_l{base_month}{last_month}'
    print(f"prefix: {prefix}")

    df = df_hist[df_hist[col_month]<=last_month]
    df = df[df[col_month]>=base_month]
    
    auth1 = df[df.authorized_flag==1]
    auth0 = df[df.authorized_flag==0]
    
    cat1_0 = False
    # cat1_0 = True
    if cat1_0:
        auth1_cat1 = auth1[auth1.category_1==1]
        auth1_cat0 = auth1[auth1.category_1==0]
        auth0_cat1 = auth0[auth0.category_1==1]
        auth0_cat0 = auth0[auth0.category_1==0]
        df_list = [ auth1_cat1 ,auth1_cat0 ,auth0_cat1 ,auth0_cat0 ]
        fname_list = ['auth1_cat1', 'auth1_cat0', 'auth0_cat1', 'auth0_cat0',]
    else:
        df_list = [auth1, auth0]
        fname_list = ['auth1', 'auth0']

    for df, base_fname in zip(tqdm(df_list), fname_list):
        fname = f'org_lag{base_month}_{last_month}_' + base_fname
        print(f"fname: {fname}")
        
        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        train_test = pd.concat([df_train, df_test], axis=0)
        train_test.set_index(key, inplace=True)
        
#         cnt_cols = ['city_id', 'state_id', 'subsector_id', 'installments']
#         cnt_cols = ['merchant_category_id']

#         # Null Count
#         tmp = df[df.isnull().any(axis=1)]
#         df_nan = pd.concat([tmp[key], (~(tmp==tmp)).sum(axis=1)], axis=1).rename(columns={0:f'{fname}_null_cnt'})
#         df_nan = df_nan.groupby(key)[f'{fname}_null_cnt'].sum()
#         train_test = train_test.join(df_nan)
#         null_cols = [col for col in train_test.columns if col.count('null_')]
#         train_test[null_cols].fillna(0, inplace=True)
        
        # 個別featureのNULLカウント
        cnt_cols = ['category_2', 'category_3', 'merchant_category_id', 'city_id', 'state_id', 'subsector_id', 'installments']
        for col in cnt_cols:
            df[col] =  df[col].replace(-1, np.nan)
            tmp = df[[key, col]]
            tmp = tmp[tmp.isnull().any(axis=1)]
            df_nan = pd.concat([tmp[key], (~(tmp==tmp)).sum(axis=1)], axis=1).rename(columns={0:f'{fname}_{col}_null_cnt'})
            df_nan = df_nan.groupby(key)[f'{fname}_{col}_null_cnt'].sum()
            train_test = train_test.join(df_nan)
            
        null_cols = [col for col in train_test.columns if col.count('null_')]
        train_test[null_cols].fillna(0, inplace=True)

        # Null_cnt Feature
        null_cols = [col for col in train_test.columns if col.count('null_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
        train_test['all_null_cnt'] = train_test[null_cols].sum(axis=1)
        
        null_cols = [col for col in train_test.columns if col.count('auth1_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
        train_test['auth1_null_cnt'] = train_test[null_cols].sum(axis=1)
        
        null_cols = [col for col in train_test.columns if col.count('auth0_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
        train_test['auth0_null_cnt'] = train_test[null_cols].sum(axis=1)
        
        null_cols = [col for col in train_test.columns if col.count('new_') and ( col.count('city') or col.count('category_3') or col.count('merchant') or col.count('install') )]
        train_test['new_null_cnt'] = train_test[null_cols].sum(axis=1)
        
        # Saving
        ignore_features = ['first_active_month', 'card_id', target]
        
        for col in train_test.columns:
            if (col.count('state')):continue
            if (col.count('subsec')):continue
            if not(col.count('null_')):continue
            length = len(train_test[col].dropna())
            if length/len(train_test)<0.1:
                print(col)
                continue
        
            train_test[col].fillna(0, inplace=True)
            feature = train_test[col].values.astype('float32')
            if feature.shape[0]!=325540:
                print(feature.shape)
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
#             utils.to_pkl_gzip(path = f'../features/2_second_valid/{prefix}_{col}@', obj=feature)

  0%|          | 0/12 [00:00<?, ?it/s]

prefix: 177_l13



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 100.80it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 168.12it/s]

fname: org_lag1_3_auth1
org_lag1_3_auth1_category_3_null_cnt
org_lag1_3_auth1_merchant_category_id_null_cnt
org_lag1_3_auth1_installments_null_cnt



 50%|█████     | 1/2 [00:08<00:08,  8.22s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 93.80it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 166.98it/s]

...


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 103.87it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 174.69it/s]

fname: org_lag10_12_auth1
org_lag10_12_auth1_category_2_null_cnt
org_lag10_12_auth1_category_3_null_cnt
org_lag10_12_auth1_merchant_category_id_null_cnt
org_lag10_12_auth1_city_id_null_cnt
org_lag10_12_auth1_installments_null_cnt



 50%|█████     | 1/2 [00:03<00:03,  3.82s/it]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 95.98it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 170.17it/s]

fname: org_lag10_12_auth0
org_lag10_12_auth0_category_2_null_cnt
org_lag10_12_auth0_category_3_null_cnt
org_lag10_12_auth0_merchant_category_id_null_cnt
org_lag10_12_auth0_city_id_null_cnt
org_lag10_12_auth0_installments_null_cnt



 83%|████████▎ | 10/12 [01:55<00:20, 10.32s/it]